### IMPORTANT! BEFORE YOU START:

### Open SuperCollider and evaluate the following line of code:

<code>FoxDot.start</code>

### Once SuperCollider indicates that it is listening for messages from FoxDot, run the following two cells:

In [1]:
from FoxDot import *

Error sending message to SuperCollider server instance: make sure FoxDot quark is running and try again.
Error: No connection made to SuperCollider server instance.


In [2]:
from __future__ import division
import numpy
import matplotlib

### Note: you only need to explicitly import the FoxDot library in environments other than the FoxDot and Troop applications

<hr>

# INTRODUCTION TO ALGORITHMS AS ART

## *>> Objective:  Create a layered rhthym section using only a sequence of binary numbers*

## First thing's first...

### Set some global parameters:

In [6]:
# Sets global tempo
Clock.bpm = 118

# Sets default scale
Scale.default.set("minorPentatonic", tuning = Tuning.just)

P[0, 3, 5, 7, 10]

### Tip: come back later and experiment with different scales and bpm values.  The same patterns can sound strikingly different when generated with a different scale, tuning, or tempo. 

In [4]:
print(Scale.names())

['aeolian', 'altered', 'bebopDom', 'bebopDorian', 'bebopMaj', 'bebopMelMin', 'blues', 'chinese', 'chromatic', 'custom', 'default', 'diminished', 'dorian', 'dorian2', 'egyptian', 'freq', 'halfDim', 'halfWhole', 'harmonicMajor', 'harmonicMinor', 'hungarianMinor', 'indian', 'justMajor', 'justMinor', 'locrian', 'locrianMajor', 'lydian', 'lydianAug', 'lydianDom', 'lydianMinor', 'major', 'majorPentatonic', 'melMin5th', 'melodicMajor', 'melodicMinor', 'minMaj', 'minor', 'minorPentatonic', 'mixolydian', 'phrygian', 'prometheus', 'romanianMinor', 'susb9', 'wholeHalf', 'wholeTone', 'yu', 'zhi']


### Make sure SuperCollider and FoxDot are communicating by creating a 'play' object:

In [5]:
# A kick-drum pulse
d1 >> play("x", sample = 3)

<d1 - play2>

### Ok, that's enough of that...

In [ ]:
# Stops only the player object
d1.stop()

<hr >

# Binary Patterns

## Thue-Morse sequence

https://en.wikipedia.org/wiki/Thue%E2%80%93Morse_sequence

### Thue-Morse sequence in Pseudo-code:

<pre><code><b>thue_morse</b>(seq_length):
    <b>input:</b> integer <i>value</i> 
    value = 0
    <b>for</b> n = 0 to seq_length - 1 by 1:     
        x = n ^ (n - 1)                         
        <b>if</b> ((x ^ (x>>1)) & 0x55555555):
            value = 1 - value
        <b>return</b> value</code></pre>

### ...or, more simply, as axiom-substitution:

<pre><code><b>Variables:</b> 0,1
<b>Start:</b> 0
<b>Rules:</b> (0 => 01), (1 => 10)</code></pre>

### A compressed implementation in Python:

In [ ]:
def thue_morse(n):
    return Pattern([bin(i).count('1') % 2 for i in range(n)])

In [ ]:
tm_seq = thue_morse(256)

print(tm_seq)

## *>> Challenge: what can we do with only 1s and 0s?*

### A simple algorithm:  

<pre><code><b>syncopated_beat</b>(<i>bin_seq</i>, <i>p</i>):
    <b>input:</b> sequence of binary digits <i>bin_seq</i>
    <b>variables:</b> pulse <i>p</i>
    <b>for</b> i in <i>bin_seq</i>:
        <b>if</b> i == 0:
            play(silence(<i>p</i>))
        <b>else</b>:
            play(sound(<i>p</i>))<code></pre>

### The above pseudo-code traverses a sequence of binary digits and either triggers a sound event if encountering a 1 or leaves a rest if encountering a 0 

### When passed as the 'amplify' parameter in a 'play' object, this generates a simple syncopated beat:

In [ ]:
# Resets the session clock back to zero so that we can hear the pattern from the beginning
#Clock.reset()

# We could also use the 'amp' parameter but more on that later...
d1 >> play("x", amplify = tm_seq, sample = 3)

### Not the most interesting beat but we can work with this...

In [ ]:
d1.stop()

### ...but it might be better to start the kick pattern on a downbeat. Let's rotate the seqence one space to the left:

In [ ]:
print("Original  => ", tm_seq)

tm_seq = tm_seq.rotate()

print(".rotate() => ", tm_seq)

### How does that sound?

In [ ]:
Clock.reset()

d1 >> play("x", amplify = tm_seq, sample = 3)

In [ ]:
d1.stop()

### That's better.

## *>> Challenge:  how can we build on the kick pattern using only this binary sequence?*

### We could 'flip' all the 0s into 1s and all the 1s into 0s 

In [ ]:
# The .invert() function creates an inverted version of pattern by subtracting its values 
# from the largest value in the pattern such that the largest value in the pattern becomes 
# the smallest (and vice versa) and the difference between other values and the min/max are swapped:
print("Original  => ", tm_seq)
print(".invert() => ", Pattern([round(x) for x in tm_seq.invert()]))

### Lace the original pattern with its inversion to construct a filled-out rhythmic framework

In [ ]:
#Clock.reset()

#d1 >> play("x", amplify = tm_seq, sample = 3)

d2 >> play("(h(H*))", amp = 1/2, amplify = tm_seq.invert(), pan = [-1,1], sample = 0)

### Now we're getting somewhere.  What else can we do?

### Scaling is a simple but powerful means of creating multi-layered patterns based on a single sequence.  
### Compress the duration by passing a fractional value to the 'dur' parameter.   
### Now we have a hi-hat pattern:

In [ ]:
# Note to musicians: in FoxDot, the '1/4' argument value here is NOT the same thing as a conventional quater-note
# We'll get into what that is later...
d3 >> play("--(-:)-", dur = 1/4, amp = expvar([1/2,1],8), amplify = tm_seq, pan = [1,-1])

### We can also rotate the sequence every n-number of beats to break up the monotony:

In [ ]:
d3.amplify = tm_seq.rotate(var(P[:int(len(tm_seq))],16))

### Random 'stutter' effects can add interest and relief from repetition:

In [ ]:
# Hi-hats
d3.every(7.5, 'stutter', 8, dur = 1/2, pshift = PRand(5), hpf = 4500)

In [ ]:
# Snaps / Claps / Shakers
#d2.sometimes('stutter', 3, dur = 1, hpf = 4500)

d2.echo = var([0,1/4],15.5)
d2.echotime = var([0,1],15.5)

In [ ]:
# Kick drum
d1.every(15.5, 'stutter', 2, dur = 1/2)

### Iterating through the sample folder creates additional timbre-variation

In [ ]:
# Kick
d1.sample = [3,[2,4]] # nested patterns

# Snaps, Claps, Shakers
d2.sample = PRand(8) # random selections

# Hi-hats
d3.sample = P[:6] # simple iteration

### Even though we're working with a binary-sequence, we don't need to limit ourselves to strict 'ON / OFF' commands.
### We can modify our algorithm to interpret 0s as 'unaccented' notes and 1s as 'accented' notes: 

In [ ]:
(tm_seq + 1)*.5

### Notice that the 1s are still 1s but the 0s have been scaled-up to 0.5s

### This 'normalized' variation makes for an accented bassline:

In [ ]:
var.bass_line = var([1,[0,3]],8)

b1 >> bass(var.bass_line, delay = PRand([.25,.75]), dur = 1/2, sus = b1.dur/2, blur = 1, amp = 1, amplify = tm_seq.rotate(1), dist = 0, hpf = 80, lpf = 900)

### ...and the same pattern an octave higher to make a synth line:

In [ ]:
s1 >> zap(b1.degree[0], oct = 5, dur = 1/4, sus = 1, amp = linvar([1/2,1],8), amplify = (tm_seq.rotate(1).invert() + 1)*.65, pan = [-1,1]).spread()

### Fill out the harmony with some pads:

In [ ]:
s2 >> swell(dur = 2, sus = 4, blur = 2, amp = linvar([1/2,1],8), amplify = 1, lpf = 1250).follow(s1).spread() + (0,2,3)

s3 >> varsaw(dur = 1/2, sus = 4, blur = 2, amp = expvar([1/2,1],8), amplify = var([0,1/2],[24,8]), hpf = 1250).follow(s2).spread()

### Add some reverb to blend everything together...

In [ ]:
# Percussion:
d1.room = .1; d1.mix = .1
d2.room = .3; d2.mix = .3
d3.room = .2; d3.mix = .2

# Synths:
s_all.room = .6; s_all.mix = .5

# Bass:
b1.room = .5; b1.mix = .3

### Add some effects for color...

### Tip: experiment with evaluating these cells in different orders and combinations

In [ ]:
# Vibrato
s1.vib = 12

In [ ]:
# "Chop" or "slicing"
s3.chop = var([0,[[8,24],16]],[24,8])

In [ ]:
# Shape distortion
s2.shape = .15

In [ ]:
# Distortion
b1.dist = linvar([0,.5],16)

In [ ]:
# Bit-crushing
d1.crush = var([4,8],8); d1.bits = 8

In [ ]:
# Pitch-shifting
d2.pshift = (-1/32,1/32)

In [ ]:
# Echo
d3.echo = 1/4

### Filter and envelope moduation for dynamic expression...

In [ ]:
# Low-pass filter
b1.lpf = linvar([5_500,1_500],16)

In [ ]:
# Blur (similar to 'release')
b1.blur = linvar([1,2],32)

In [ ]:
# Blur
s_all.blur = linvar([1/2,4],32)

In [ ]:
# Band-pass filter
s1.bpf = linvar([2_500,1_200],16)
# Band-pass filter resonance
s1.bpr = linvar([1,.5],8)

In [ ]:
# High-pass filter
d3.hpf = linvar([4_000,900],16)

### Yes, it's easy to get carried away... 
### Let's bring it down for a minute...

In [ ]:
# Assign multiple player objects to a single object
ens_1 = Group(d1,d2,b1,s2)

# Applies effect to a group
ens_1.solo()

# Applies effect(s) the entire session
Master().amplify = linvar([.5,1],32)
Master().lpf = linvar([1_100,250],32,start=now)

In [ ]:
Master().amplify = 1
Master().lpf = linvar([250,20_000],[8,inf],start=now)

In [ ]:
d1 >> play("P", amp = 2/3, amplify = tm_seq, lpf = 900, crush = 0, pshift = 0, sample = PWalk())

In [ ]:
d2 >> play("m", amp = 1/2, amplify = tm_seq.invert(), pan = PRand([-1,1]), mix = .4, pshift = 0, sample = P[:10])

In [ ]:
d3 >> play("ss(s+)s", dur = 1/4, amp = 1/2, amplify = tm_seq, pan = [-1,1], sample = P[:20], mix = .2)

In [ ]:
b1 >> bass(var.bass_line, delay = PRand([.25,.5,.75,1.5]), dur = 1/2, sus = b1.dur/2, amp = 1, amplify = tm_seq.rotate(1), lpf = 1_800)

In [ ]:
s1 >> piano(var.bass_line, oct = 4, dur = 1/4, sus = s1.dur*2, blur = 2, amp = linvar([1/3,1],8), amplify = (tm_seq.rotate(1).invert() + 1)*.65, pan = 0).spread() + (0,2,3)

In [ ]:
s2 >> bell(P[0,1,2].arp([0,[3,-2],[4,1,5]]), oct = (7,5,6), delay = PRand([0,.25,.5,.75,1.25,1.5]), dur = 1/2, sus = 2, blur = 2, chop = 0, amp = linvar([1/2,1],8), amplify = tm_seq*.2, pan = linvar([1,-1],5))

In [ ]:
Clock.clear()

### Let's add a new sequence to the mix that uses more than just 0's and 1's...

# Integer Sequences

## Rudin-Shapiro sequence:

https://en.wikipedia.org/wiki/Rudin%E2%80%93Shapiro_sequence

[: PSEUDO-CODE

### Rudin-Shapiro sequence implemented in Python:

In [ ]:
def rudin_shapiro(n):
    
    def hamming(x):
        # Hamming weight of a binary sequence
        return bin(x).count('1')

    out = []
    for i in range(n):
        b = hamming(i << 1 & i)
        a = (-1)**b
        out.append(a)
        
    pat = P[0]
    for i in range(n):
        pat.append(pat[i] + out[i])

    return Pattern(pat)

In [ ]:
rs_seq = rudin_shapiro(64)

print(rs_seq)

## >> Challenge: how can we introduce this new sequence while keeping it connected to the binary pattern we've been using?

### We could create a new sequence that is a composite of the old and new patterns...

## Composite Functions

### Using the Thue-Morse sequence as a sequence of boolean conditionals, we can create decorative melodic fragments that interlink nicely with the existing rhythm section:

In [ ]:
def composite(f,g):
    
    out = []
    k = 0
    for i in f:
        if i == 0:
            out.append(0)
        else:
            out.append(g[k])
        k += 1
    
    return Pattern(out)

In [ ]:
fg_seq = composite(thue_morse(64),rudin_shapiro(64).invert())

print(fg_seq.transform(int))

In [ ]:
A = P[36+8,16+8]
B = A.reverse()

f1 >> pluck(fg_seq, dur = var([2,1/2],A), sus = f1.dur, blur = 3, amp = 1/2, amplify = tm_seq, vib = var([2,0],A))

f2 >> nylon(fg_seq, dur = var([1/2,2],B), sus = f2.dur, blur = 3, amp = 1/2, amplify = (tm_seq.invert() + 1)*.25)

f1.pan = [-1,1]
f2.pan = [1,-1]

f_all.room = .7
f_all.mix = .4

TODO: ADD TEMPORAL SCHEDULING

In [ ]:
Group(f1,f2,s1,s2,d1).solo()

In [ ]:
Group(f1,f2,s2,d1).solo()

In [ ]:
f_all.solo()

### Ok that's enough for now...

In [ ]:
# Stops the entire session
Clock.clear()

# 'Generative' Systems

## Per Nørgård "Infinity Series"

### The *n*th term of Nørgård's sequence is *s*(*n*); it specifies how far away the *n*th element is, in index position, from the first element (located at index 0).

#### The *n*th value in the Infinity Series can be found using the rules:

<pre><code><b>norgard</b>(<i>n</i>):
    <b>input:</b> <i>n</i>-th value of the series <i>n</i>
    <b>if</b> <i>n</i> == 0:
        return 0
    <b>elif</b> <i>n</i> is even:
        return -1 * <b>norgard</b>(<i>n</i> / 2)
    <b>elif</b> <i>n</i> is odd:
        return <b>norgard</b>((<i>n</i> - 1) / 2) + 1</code></pre>

### Nørgård's "Infinity Series" implemented in Python:

In [ ]:
def norg(n):
    pn = [0]*n
    pn[0] = 0
    pn[1] = 1

    for i in range(1,int(n/2)):
        pn[2*i] = pn[2*i-2] - (pn[i]-pn[i-1])
        pn[2*i+1] = pn[2*i-1] + (pn[i]-pn[i-1])

    return Pattern(pn)

In [ ]:
norg_seq = norg(128)
tm_seq = thue_morse(64)

print(norg_seq)

In [ ]:
tb >> play("P", dur = 1/3, amp = linvar([1/2,1],4), amplify = tm_seq, pan = PRand(-1,1), room = .4, mix = .2)

In [ ]:
dd >> play("R", dur = 4, amp = linvar([1/2,1],36), room = .6, mix = .4)

In [ ]:
tm >> play("m", dur = 2/3, amp = (tm_seq.invert() + 1)*.5, amplify = linvar([1/4,1],18), sample = P[:1], room = .5, mix = .4)

In [ ]:
gg >> play("Q", dur = 18, echo = 3/5, echotime = 18, pshift = norg_seq, amp = 2/3, room = 1, mix = [.5,.7,1]).spread()

In [ ]:
tb.sample = norg_seq

In [ ]:
dd.pshift = norg_seq

In [ ]:
f_all.stop()

In [ ]:
s1 >> viola(norg_seq, oct = 6, dur = 7, sus = s1.dur, blur = 3, echo = 2/3, echotime = 9, amp = 1/2, pan = PWhite())

In [ ]:
s2 >> klank(norg_seq.invert(), oct = 5, dur = 9, sus = s2.dur, blur = 3, echo = 2/3, echotime = 9, amp = 1/2).spread()

In [ ]:
s3 >> varsaw(norg_seq.mirror(), oct = 4, dur = 5, sus = s3.dur, blur = 2, shape = .2).spread()

In [ ]:
s4 >> dub(norg_seq, dur = 18, sus = s4.dur, blur = 2, amp = 1/3, dist = .1).spread()

In [ ]:
gl >> glass(norg_seq.splice(norg_seq.invert()), oct = (5,4,3,6), dur = PRand(7,18), sus = gl.dur, blur = 2, shape = .2, pan = [1,-1]).spread()

In [ ]:
vx >> charm(norg_seq, root = PRand([-1/3,-1/6,-9/8,-10/9,0]), oct = 6, dur = PSum([[7,[11,13]],[5,[4,9],3]],18), sus = vx.dur, blur = 2, pan = [1,-1])

In [ ]:
cr >> creep(norg_seq.mirror(), oct = (4,5), dur = 36, vib = 5, slidefrom = norg_seq, room = 1, mix = .7, amp = 1/3, echo = 3/5, echotime = 18, pan = [-1,1])

In [ ]:
ens_1 = Group(tb,dd,tm,gg)
ens_2 = Group(s1,s2,g1)
ens_3 = Group(s3,s4,cr)

In [ ]:
ens_2.lpf = linvar([1200,350],36)
ens_2.lpr = linvar([.1,1],11)

In [ ]:
ens_3.hpf = linvar([350,900],36)
ens_3.hpr = linvar([.1,1],7)

In [ ]:
ens_1.bpf = linvar([1100,2500],36)
ens_1.bpr = linvar([1,.1],9)

In [ ]:
all_ens = Group(ens_1,ens_2,ens_3)

In [ ]:
all_ens.room = 1
all_ens.mix = linvar([.1,1],54)

In [ ]:
all_ens.spin = linvar([1/6,9],72)

In [ ]:
all_ens.echo = 4/5
all_ens.echotime = 9

In [ ]:
Master().room = 1
Master().mix = .5

In [ ]:
Clock.clear()

## Axiom-Substitution

## Lindenmayer Systems (L-Systems)

In [ ]:
def algea(n):
    axioms = {'A':'AB','B':'A'}
    seq = 'A'
    for i in range(n):
        out = ""
        for ch in seq:
            out += axioms[ch]
        seq = out
    
    return seq

In [ ]:
print(algea(5))